# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook.

In [1]:
from pprint import pprint
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from wombat.core import Simulation
from wombat.core import Metrics

%matplotlib inline
pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

In [2]:
simulation_name = "example_dinwoodie"
library_path = Path("../../../../wombat/library/dinwoodie/").resolve()

sim = Simulation(simulation_name, library_path, "base.yaml")
sim.run()

# For convenience only
metrics = sim.metrics

## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 94.4%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.93,0.92,0.97,0.89,0.98,0.97,0.95,0.98,0.98,0.93,0.91,0.96,0.98,0.98,0.92,0.98,0.98,0.91,0.94,0.97,0.98,0.97,0.97,0.87,0.98,0.94,0.92,0.92,0.89,0.93,0.93,0.98,0.92,0.88,0.93,0.92,0.94,0.93,0.94,0.87,0.94,0.98,0.96,0.97,0.98,0.91,0.98,0.97,0.97,0.98,0.91,0.95,0.97,0.98,0.98,0.92,0.97,0.92,0.98,0.98,0.94,0.87,0.98,0.97,0.90,0.91,0.97,0.98,0.89,0.98,0.88,0.90,0.93,0.92,0.93,0.91,0.94,0.96,0.95,0.98


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,year,windfarm
0,2003,0.97
1,2004,0.92
2,2005,0.92
3,2006,0.95
4,2007,0.96
5,2008,0.95
6,2009,0.95
7,2010,0.93
8,2011,0.93
9,2012,0.95


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,month,windfarm
0,1,0.94
1,2,0.93
2,3,0.93
3,4,0.92
4,5,0.94
5,6,0.94
6,7,0.96
7,8,0.96
8,9,0.96
9,10,0.96


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

,year,month,windfarm
0,2003,1,0.99
1,2003,2,0.98
2,2003,3,0.98
3,2003,4,0.98
4,2003,5,0.99
5,2003,6,0.98
6,2003,7,0.99
7,2003,8,0.99
8,2003,9,0.97
9,2003,10,0.95


## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.45%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.44,0.43,0.46,0.41,0.46,0.46,0.45,0.47,0.47,0.45,0.43,0.46,0.47,0.47,0.43,0.47,0.47,0.43,0.45,0.46,0.47,0.46,0.46,0.41,0.47,0.45,0.44,0.43,0.42,0.45,0.44,0.47,0.44,0.41,0.45,0.44,0.45,0.44,0.45,0.41,0.44,0.47,0.46,0.46,0.47,0.44,0.47,0.46,0.46,0.47,0.43,0.45,0.46,0.47,0.47,0.44,0.46,0.44,0.47,0.47,0.44,0.41,0.47,0.46,0.43,0.44,0.46,0.47,0.43,0.47,0.42,0.43,0.44,0.44,0.44,0.44,0.45,0.46,0.45,0.47


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.43
2005,0.45
2006,0.44
2007,0.49
2008,0.50
2009,0.45
2010,0.39
2011,0.47


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.59
2,0.49
3,0.49
4,0.40
5,0.38
6,0.32
7,0.33
8,0.36
9,0.46


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.62
     2          0.36
     3          0.37
     4          0.53
     5          0.35
     6          0.31
     7          0.30
     8          0.30
     9          0.27
     10         0.42
     11         0.52
     12         0.52
2004 1          0.51
     2          0.47
     3          0.48
     4          0.37
     5          0.28
     6          0.36
     7          0.26
     8          0.39
     9          0.56
     10         0.54
     11         0.45
     12         0.47

## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 100%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,year,Completion Rate
0,2003,0.98
1,2004,1.01
2,2005,1.00
3,2006,1.00
4,2007,1.00
5,2008,1.00
6,2009,1.00
7,2010,1.00
8,2011,0.99
9,2012,1.01


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,year,Completion Rate
0,1,0.96
1,2,1.04
2,3,0.97
3,4,1.00
4,5,1.04
5,6,1.02
6,7,1.02
7,8,0.96
8,9,1.03
9,10,0.98


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

,year,month,Completion Rate
0,2003,1,0.86
1,2003,2,1.11
2,2003,3,0.96
3,2003,4,1.05
4,2003,5,1.02
5,2003,6,0.99
6,2003,7,1.01
7,2003,8,1.01
8,2003,9,0.96
9,2003,10,0.92


## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $482,197.01/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","2,660,000.00","93,887,500.00"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,year,equipment_cost
0,2003,"10,433,781.25"
1,2004,"11,056,250.00"
2,2005,"11,682,250.00"
3,2006,"10,819,750.00"
4,2007,"11,207,250.00"
5,2008,"10,975,000.00"
6,2009,"11,657,250.00"
7,2010,"12,982,250.00"
8,2011,"12,176,000.00"
9,2012,"12,737,500.00"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,month,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1,"543,010.42","543,010.42","543,010.42",0.00,0.00
1,2,"495,250.00","495,250.00","495,250.00",0.00,0.00
2,3,"542,500.00","542,500.00","542,500.00",0.00,0.00
3,4,"525,000.00","525,000.00","525,000.00",0.00,0.00
4,5,"542,500.00","542,500.00","542,500.00",0.00,"33,218,750.00"
5,6,"525,000.00","525,000.00","525,000.00","2,660,000.00",0.00
6,7,"542,500.00","542,500.00","542,500.00",0.00,"31,343,750.00"
7,8,"542,500.00","542,500.00","542,500.00",0.00,0.00
8,9,"525,000.00","525,000.00","525,000.00",0.00,"29,325,000.00"
9,10,"542,500.00","542,500.00","542,500.00",0.00,0.00


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

,year,month,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,2003,1,"54,760.42","54,760.42","54,760.42",0.00,0.00
1,2003,2,"49,000.00","49,000.00","49,000.00",0.00,0.00
2,2003,3,"54,250.00","54,250.00","54,250.00",0.00,0.00
3,2003,4,"52,500.00","52,500.00","52,500.00",0.00,0.00
4,2003,5,"54,250.00","54,250.00","54,250.00",0.00,"2,750,000.00"
5,2003,6,"52,500.00","52,500.00","52,500.00","266,000.00",0.00
6,2003,7,"54,250.00","54,250.00","54,250.00",0.00,"2,750,000.00"
7,2003,8,"54,250.00","54,250.00","54,250.00",0.00,0.00
8,2003,9,"52,500.00","52,500.00","52,500.00",0.00,"2,750,000.00"
9,2003,10,"54,250.00","54,250.00","54,250.00",0.00,0.00


## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,0.44,0.43,0.44,0.22,0.84


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,0.38,0.36,0.39,0.21,0.33
2004,0.64,0.65,0.61,0.36,1.00
2005,0.52,0.53,0.48,0.21,1.00
2006,0.47,0.40,0.50,0.25,0.98
2007,0.55,0.52,0.57,0.21,0.49
2008,0.51,0.44,0.37,0.21,0.96
2009,0.38,0.33,0.37,0.32,0.73
2010,0.33,0.37,0.38,0.07,1.00
2011,0.34,0.39,0.35,0.21,1.00


## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [24]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [25]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [26]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,year,total_labor_cost
0,2003,0.00
1,2004,0.00
2,2005,0.00
3,2006,0.00
4,2007,0.00
5,2008,0.00
6,2009,0.00
7,2010,0.00
8,2011,0.00
9,2012,0.00


In [27]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,month,total_labor_cost
0,1,0.00
1,2,0.00
2,3,0.00
3,4,0.00
4,5,0.00
5,6,0.00
6,7,0.00
7,8,0.00
8,9,0.00
9,10,0.00


In [28]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

,year,month,total_labor_cost
0,2003,1,0.00
1,2003,2,0.00
2,2003,3,0.00
3,2003,4,0.00
4,2003,5,0.00
5,2003,6,0.00
6,2003,7,0.00
7,2003,8,0.00
8,2003,9,0.00
9,2003,10,0.00


## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
##### **NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [29]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,reason,total_cost
0,Maintenance,"2,135,000.00"
1,Repair,"24,046,333.33"
2,Mobilization,"15,000,000.00"
3,Weather Delay,"11,653,145.83"
4,No Requests,"24,075,531.25"
5,Not in Shift,"38,817,270.83"


In [30]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,reason,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
0,Maintenance,0.00,0.00,0.00,"2,135,000.00","2,135,000.00"
1,Repair,0.00,0.00,0.00,"24,046,333.33","24,046,333.33"
2,Mobilization,0.00,0.00,0.00,"15,000,000.00","15,000,000.00"
3,Weather Delay,0.00,0.00,0.00,"11,653,145.83","11,653,145.83"
4,No Requests,0.00,0.00,0.00,"24,075,531.25","24,075,531.25"
5,Not in Shift,0.00,0.00,0.00,"38,817,270.83","38,817,270.83"


In [31]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

,year,reason,total_cost
0,2003,Maintenance,"167,562.50"
1,2003,Repair,"946,000.00"
2,2003,Mobilization,"1,500,000.00"
3,2003,Weather Delay,"546,635.42"
4,2003,No Requests,"5,734,375.00"
5,2003,Not in Shift,"1,539,208.33"
6,2004,Maintenance,"335,562.50"
7,2004,Repair,"2,914,395.83"
8,2004,Mobilization,"1,500,000.00"
9,2004,Weather Delay,"1,170,135.42"


In [32]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

,month,reason,total_cost
0,1,Maintenance,"147,510.42"
1,1,Repair,"232,531.25"
2,1,Mobilization,0.00
3,1,Weather Delay,"90,052.08"
4,1,No Requests,"772,114.58"
5,1,Not in Shift,"386,822.92"
6,2,Maintenance,"150,572.92"
7,2,Repair,"196,437.50"
8,2,Mobilization,0.00
9,2,Weather Delay,"75,322.92"


In [33]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

,year,month,reason,total_cost
0,2003,1,Maintenance,0.00
1,2003,1,Repair,"18,010.42"
2,2003,1,Mobilization,0.00
3,2003,1,Weather Delay,"8,020.83"
4,2003,1,No Requests,"117,468.75"
5,2003,1,Not in Shift,"20,781.25"
6,2003,2,Maintenance,0.00
7,2003,2,Repair,"24,062.50"
8,2003,2,Mobilization,0.00
9,2003,2,Weather Delay,"6,781.25"


## Component

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
##### **NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [34]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,component,total_cost
0,cable,0.00
1,drive_train,0.00
2,electrical_system,0.00
3,electronic_control,0.00
4,gearbox,0.00
5,generator,"114,986,218.75"
6,hydraulic_system,0.00
7,mechanical_brake,0.00
8,rotor_blades,0.00
9,rotor_hub,0.00


In [35]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

,component,action,materials_cost,total_labor_cost,equipment_cost,total_cost
0,generator,delay,0,0.00,"50,468,885.42","50,468,885.42"
1,generator,maintenance,0,0.00,"2,135,000.00","2,135,000.00"
2,generator,repair,0,0.00,"24,046,333.33","24,046,333.33"


In [36]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,year,component,materials_cost,total_labor_cost,equipment_cost,total_cost
0,2003,cable,0.00,0.00,0.00,0.00
1,2003,drive_train,0.00,0.00,0.00,0.00
2,2003,electrical_system,0.00,0.00,0.00,0.00
3,2003,electronic_control,0.00,0.00,0.00,0.00
4,2003,gearbox,0.00,0.00,0.00,0.00
5,2003,generator,"2,065,000.00",0.00,"3,197,875.00","5,262,875.00"
6,2003,hydraulic_system,0.00,0.00,0.00,0.00
7,2003,mechanical_brake,0.00,0.00,0.00,0.00
8,2003,rotor_blades,0.00,0.00,0.00,0.00
9,2003,rotor_hub,0.00,0.00,0.00,0.00


In [37]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,month,component,materials_cost,total_labor_cost,equipment_cost,total_cost
0,1,cable,0.00,0.00,0.00,0.00
1,1,drive_train,0.00,0.00,0.00,0.00
2,1,electrical_system,0.00,0.00,0.00,0.00
3,1,electronic_control,0.00,0.00,0.00,0.00
4,1,gearbox,0.00,0.00,0.00,0.00
5,1,generator,"1,110,000.00",0.00,"855,385.42","1,965,385.42"
6,1,hydraulic_system,0.00,0.00,0.00,0.00
7,1,mechanical_brake,0.00,0.00,0.00,0.00
8,1,rotor_blades,0.00,0.00,0.00,0.00
9,1,rotor_hub,0.00,0.00,0.00,0.00


In [38]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

,year,month,component,materials_cost,total_labor_cost,equipment_cost,total_cost
0,2003,1,cable,0.00,0.00,0.00,0.00
1,2003,1,drive_train,0.00,0.00,0.00,0.00
2,2003,1,electrical_system,0.00,0.00,0.00,0.00
3,2003,1,electronic_control,0.00,0.00,0.00,0.00
4,2003,1,gearbox,0.00,0.00,0.00,0.00
5,2003,1,generator,"13,000.00",0.00,"45,281.25","58,281.25"
6,2003,1,hydraulic_system,0.00,0.00,0.00,0.00
7,2003,1,mechanical_brake,0.00,0.00,0.00,0.00
8,2003,1,rotor_blades,0.00,0.00,0.00,0.00
9,2003,1,rotor_hub,0.00,0.00,0.00,0.00


## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [39]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [40]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,000,080.00"


In [41]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [42]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [43]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,year,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
1,2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2,2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
3,2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
4,2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
5,2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
6,2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
7,2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
8,2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
9,2012,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [44]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"91,736.00","65,220.00","61,392.00",488.00
major repair,"120,285.26","1,498.00","1,361.00",28.00
major replacement,"262,791.42","10,605.00","262,235.42",67.00
manual reset,"145,662.83","32,579.00","12,075.00","5,528.00"
medium repair,"9,947.28","8,026.00","6,487.00",178.00
minor repair,"58,448.24","28,246.00","15,780.00","2,233.00"


## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [45]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,461,478,693.00"


In [46]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,461,478,693.00","115,551,938.50","114,341,830.50","121,422,445.50","108,851,127.00","121,777,663.50","121,929,734.50","118,602,893.00","122,368,605.50","123,359,828.00","117,453,404.50","112,950,150.00","120,932,158.50","122,622,303.50","122,620,763.50","113,803,339.50","122,729,919.50","122,476,242.50","113,284,261.50","119,201,340.00","121,759,697.50","122,561,701.00","121,978,412.00","121,535,217.50","108,593,641.00","123,255,241.00","117,460,723.50","115,429,706.50","114,166,581.50","110,945,921.50","117,377,747.50","116,803,423.50","122,727,192.50","115,525,103.00","108,414,793.00","117,931,163.00","115,276,550.50","118,493,084.50","116,604,869.00","117,797,983.50","107,993,933.50","115,849,666.00","122,453,687.50","121,199,192.00","122,094,071.00","123,279,483.50","115,006,761.00","122,382,580.50","121,432,822.00","121,985,058.50","122,330,301.50","113,747,088.50","119,486,651.50","121,547,864.00","122,605,622.50","123,133,407.00","115,131,168.00","121,976,245.00","114,763,210.50","122,476,808.50","122,849,303.00","116,016,117.50","109,128,613.00","122,582,703.50","122,284,170.50","113,211,356.00","116,085,288.00","121,821,535.50","122,624,437.00","112,613,069.00","122,548,926.00","109,674,313.00","112,807,473.50","116,696,688.00","115,109,682.50","116,278,271.50","116,401,393.00","118,403,644.50","121,267,740.50","118,912,000.50","122,367,642.50"


In [47]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"854,383,612.50"
2004,"904,945,977.00"
2005,"950,712,982.00"
2006,"933,429,767.00"
2007,"1,025,838,720.50"
2008,"1,045,723,865.50"
2009,"947,254,396.00"
2010,"830,190,743.50"
2011,"989,537,969.00"


In [48]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,048,708,513.50"
2,"792,533,269.00"
3,"870,838,364.50"
4,"682,996,990.50"
5,"671,101,288.00"
6,"547,942,556.00"
7,"585,076,852.00"
8,"637,278,539.50"
9,"796,644,115.00"


In [49]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     111,033,171.50
     2      58,152,558.50
     3      66,715,100.00
     4      91,506,438.50
     5      62,231,291.00
     6      53,152,561.50
     7      53,688,794.00
     8      53,635,704.50
     9      47,076,236.50
     10     74,177,452.00
     11     90,309,585.00
     12     92,704,719.50
2004 1      91,244,305.00
     2      79,003,535.50
     3      85,102,807.00
     4      63,304,236.50
     5      49,854,888.50
     6      63,049,813.00
     7      46,862,458.50
     8      70,381,218.00
     9      97,041,657.50
     10     97,167,461.00
     11     78,563,552.00
     12     83,370,044.50

## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> A PySAM file needs to be defined in your library's windfarm directory. This part of the analysis is highly dependent on the inputs provided, and there is not a validated input data sheet used, this is merely a functionality demonstration.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [50]:
npv = metrics.pysam_npv()
print(f"NPV: ${npv:,.0f}")

NPV: $31,785,832


### Real Levelized Cost of Energy (LCOE)

In [51]:
lcoe = metrics.pysam_lcoe_real()
print(f"Real LCOE: ${lcoe:,.2f}/kW")

Real LCOE: $-0.00/kW


### Nominal Levelized Cost of Energy (LCOE)

In [52]:
lcoe = metrics.pysam_lcoe_nominal()
print(f"Nominal LCOE: ${lcoe:,.2f}/kW")

Nominal LCOE: $-0.01/kW


### After-tax Internal Return Rate (IRR)

In [53]:
npv = metrics.pysam_irr()
print(f"IRR: {npv:,.1f}%")

IRR: 14.3%


### One Data Frame to Rule Them All

In [54]:
metrics.pysam_all_outputs()

,Value
Metric,
After Tax NPV ($),"31,785,832.48"
Nominal LCOE ($/kW),-0.01
Real LCOE ($/kW),-0.00
After Tax IRR (%),14.29


In [55]:
sim.env.cleanup_log_files(log_only=False)